RMSNorm 去中心化(即不使用β参数)的影响:
1. 减少了模型参数，降低了计算复杂度
2. 可能提高模型的泛化能力，因为减少了与特定任务相关的先验假设
3. 在T5等模型中的成功应用表明，对于某些任务，中心化操作可能不是必要的

1. **LayerNorm**:
   - 同时进行中心化(减去均值)和标准化(除以标准差)
   - 包含两个可学习参数γ和β
   - 计算成本略高于RMSNorm

2. **RMSNorm**:
   - 仅进行标准化(除以均方根)
   - 通常只使用一个可学习参数γ
   - 具有尺度不变性，梯度与输入尺度成反比
   - 计算效率更高，在某些任务中表现与LayerNorm相当

3. **选择建议**:
   - 当输入分布已经接近零均值时，RMSNorm可能是更高效的选择
   - 对于需要强归一化的情况，LayerNorm可能更合适
   - 实际应用中可以通过实验确定哪种归一化更适合特定任务

在计算资源受限或模型很大时优先考虑 RMSNorm

当不确定哪种更好时，可以在验证集上进行小规模实验

一些现代架构如 LLAMA 已经采用 RMSNorm 替代 LayerNorm

两者可以混合使用，如关键层使用 LayerNorm，其他层使用 RMSNorm



RMSNorm具有尺度不变性，即输入乘以一个标量，输出结果不变(除了缩放参数γ的影响)。

RMSNorm的梯度与输入尺度成反比，这是其数值稳定性的重要特性。

In [1]:
import torch
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))  # 可学习的缩放参数

    def _norm(self, x: torch.Tensor):
        # x: (batch, seq_len, dim)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x: torch.Tensor):
        return self.weight * self._norm(x)

In [2]:
class LayerNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))  # 缩放参数
        self.bias = nn.Parameter(torch.zeros(dim))    # 偏置参数

    def forward(self, x: torch.Tensor):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased=False, keepdim=True)
        return self.weight * (x - mean) / torch.sqrt(var + self.eps) + self.bias

In [3]:
# 测试数据
batch, seq_len, dim = 2, 5, 8
x = torch.randn(batch, seq_len, dim)

# 初始化两种归一化
rms_norm = RMSNorm(dim)
ln_norm = LayerNorm(dim)

# 前向传播
rms_out = rms_norm(x)
ln_out = ln_norm(x)

print("输入形状:", x.shape)
print("RMSNorm 输出形状:", rms_out.shape)
print("LayerNorm 输出形状:", ln_out.shape)

# 尺度不变性测试
x_scaled = x * 10.0
rms_scaled = rms_norm(x_scaled)
ln_scaled = ln_norm(x_scaled)

print("\n尺度不变性测试:")
print("RMSNorm 原始输出与缩放输出比例:", (rms_scaled / rms_out).mean().item())
print("LayerNorm 原始输出与缩放输出比例:", (ln_scaled / ln_out).mean().item())

输入形状: torch.Size([2, 5, 8])
RMSNorm 输出形状: torch.Size([2, 5, 8])
LayerNorm 输出形状: torch.Size([2, 5, 8])

尺度不变性测试:
RMSNorm 原始输出与缩放输出比例: 1.0000005960464478
LayerNorm 原始输出与缩放输出比例: 1.0000005960464478


In [4]:
import timeit

# 计时测试
def time_normalization(norm, x):
    def fn():
        return norm(x)
    return timeit.timeit(fn, number=100000)

rms_time = time_normalization(rms_norm, x)
ln_time = time_normalization(ln_norm, x)

print("\n计算效率对比 (1000次前向传播):")
print(f"RMSNorm: {rms_time:.4f} 秒")
print(f"LayerNorm: {ln_time:.4f} 秒")
print(f"速度提升: {(ln_time - rms_time)/ln_time:.1%}")


计算效率对比 (1000次前向传播):
RMSNorm: 0.7783 秒
LayerNorm: 1.1759 秒
速度提升: 33.8%


In [5]:
# 梯度测试
x.requires_grad_(True)
target = torch.randn_like(x)

# RMSNorm 梯度
rms_out = rms_norm(x)
rms_loss = (rms_out - target).pow(2).mean()
rms_loss.backward()
rms_grad = x.grad.clone()
x.grad.zero_()

# LayerNorm 梯度
ln_out = ln_norm(x)
ln_loss = (ln_out - target).pow(2).mean()
ln_loss.backward()
ln_grad = x.grad.clone()

print("\n梯度行为对比:")
print("RMSNorm 梯度均值:", rms_grad.mean().item())
print("LayerNorm 梯度均值:", ln_grad.mean().item())
print("RMSNorm 梯度方差:", rms_grad.var().item())
print("LayerNorm 梯度方差:", ln_grad.var().item())


梯度行为对比:
RMSNorm 梯度均值: 0.004218118265271187
LayerNorm 梯度均值: -2.3283064365386963e-10
RMSNorm 梯度方差: 0.0006827886682003736
LayerNorm 梯度方差: 0.0006636029575020075
